# Counting experiment example

In this example we do not use a shape analysis but instead do a counting experiment, i.e., we know the total number of events after some selection *Nobs* and we have an estimation of the number of background events *Nbkg*. The goal is to determine the number of signal events, *Nsig*, such that

$$ Nobs = Nbkg + Nsig $$

and do inferences on this parameter of interest.

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import zfit
from utils import plotlimit
from zfit.loss import UnbinnedNLL
from zfit.minimize import Minuit

from hepstats.hypotests import Discovery, UpperLimit
from hepstats.hypotests.calculators import AsymptoticCalculator, FrequentistCalculator
from hepstats.hypotests.parameters import POI, POIarray

We define the three yields used in the analysis:

In [ ]:
Nsig = zfit.Parameter("Nsig", 0, -100.0, 100)
Nbkg = zfit.Parameter("Nbkg", 100, 0, 500)
Nobs = zfit.ComposedParameter("Nobs", lambda a, b: a + b, params=[Nsig, Nbkg])

We assume *Nobs* is Poisson distributed. In the cell below we define the Poisson PDF ourselves because it is not yet available in `zfit`. 

In [ ]:
obs = zfit.Space("N", limits=(0, 800))
model = zfit.pdf.Poisson(obs=obs, lamb=Nobs)

In this example the number of events in the dataset is 370, and the estimated number of background events is 340 which means the number of signal events is 30.

In [ ]:
n = 370
nbkg = 340

data = zfit.data.Data.from_numpy(obs=obs, array=np.array([n]))
Nbkg.set_value(nbkg)
Nbkg.floating = False

# likelihood function function
nll = UnbinnedNLL(model=model, data=data)

# Instantiate a minuit minimizer
minimizer = Minuit(verbosity=0)

# minimisation of the loss function
minimum = minimizer.minimize(loss=nll)

Using `hepstats` one can determine if this excess of signal is significant or not.

### Inferences with the `AsymptoticCalculator`

In [ ]:
# instantation of the calculator
calculator = AsymptoticCalculator(nll, minimizer)
calculator.bestfit = minimum  # optionnal

#### Discovery test:

In [ ]:
discovery_test = Discovery(calculator, POI(Nsig, 0))
pnull, significance = discovery_test.result()

The significance of the signal excess is **1.6 sigma**.

#### Upper limit:

In [ ]:
# parameter of interest to scan
poi_scan = POIarray(Nsig, np.linspace(0.0, 100, 20))
# parameter of interest set at the background only hypothesi
poi_bkg_only = POI(Nsig, 0)

In [ ]:
# instantation of the discovery test
ul = UpperLimit(calculator, poi_scan, poi_bkg_only)
ul.upperlimit(alpha=0.05, CLs=False)

f = plt.figure(figsize=(9, 8))
plotlimit(ul, alpha=0.05, CLs=False)
plt.xlabel("Nsig");

#### Upper limit with uncertainty on the background prediction:

Let's assume for now that the estimation of the background yield is 
$$ Nbkg = 340 \pm 25.$$
We add a Gaussian constraint on the likelihood and let *Nbkg* float.

In [ ]:
nbkg_constr = zfit.constraint.GaussianConstraint(params=Nbkg, observation=340, sigma=25)
nll.add_constraints(nbkg_constr)
Nbkg.floating = True

In [ ]:
# instantation of the calculator
calculator = AsymptoticCalculator(nll, Minuit(verbosity=0))

# instantation of the discovery test
poi_scan = POIarray(Nsig, np.linspace(0.0, 150, 20))
ul = UpperLimit(calculator, poi_scan, poi_bkg_only)
ul.upperlimit(alpha=0.05, CLs=False)

f = plt.figure(figsize=(9, 8))
plotlimit(ul, alpha=0.05, CLs=False)
plt.xlabel("Nsig");

### Inferences with the `FrequentistCalculator`

In [ ]:
nll = UnbinnedNLL(model=model, data=data)
Nbkg.set_value(nbkg)
Nbkg.floating = False

calculator = FrequentistCalculator(nll, minimizer, ntoysalt=1000, ntoysnull=1000)
calculator.bestfit = minimum  # optional

#### Discovery test:

In [ ]:
discovery_test = Discovery(calculator, POI(Nsig, 0))
pnull, significance = discovery_test.result()

#### Upper limit:

In [ ]:
# parameter of interest to scan
poi_scan = POIarray(Nsig, np.linspace(0.0, 100, 20))
# parameter of interest set at the background only hypothesi
poi_bkg_only = POI(Nsig, 0)

# instantation of the discovery test
ul = UpperLimit(calculator, poi_scan, poi_bkg_only)
ul.upperlimit(alpha=0.05, CLs=False)

f = plt.figure(figsize=(9, 8))
plotlimit(ul, alpha=0.05, CLs=False)
plt.xlabel("Nsig");